In [ ]:
import os
from keras.applications import MobileNet
from keras.applications import VGG16
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn import preprocessing
from keras.utils import np_utils

In [ ]:
train_meta_df = pd.read_csv('../input/meta.csv')
test_meta_df = pd.read_csv('../input/test_meta.csv')

NUM_TRAIN = len(train_meta_df)
NUM_TEST = len(test_meta_df)
IMG_SIZE = 224
NUM_BATCH = 128
NUM_EPOCH = 40

X_train = np.memmap(
    filename='../input/X_train.npy', dtype=np.float16, 
    mode='r', shape=(NUM_TRAIN, IMG_SIZE, IMG_SIZE, 3) )
y_train = train_meta_df['class'].values

le = preprocessing.LabelEncoder()
le.fit(y_train)
NUM_CLASSES = len(le.classes_)
y_train = le.transform(y_train)

X_test = np.memmap(
    filename='../input/X_test.npy', dtype=np.float16, 
    mode='r', shape=(NUM_TEST, IMG_SIZE, IMG_SIZE, 3))
y_test = test_meta_df['class'].values
y_test = le.transform(y_test)
y_test = np_utils.to_categorical(y=y_test, num_classes=NUM_CLASSES)

#  MobileNet
model = MobileNet(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    alpha=1.0, depth_multiplier=1, weights=None,
    classes=NUM_CLASSES)

model.compile(
    loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(
    x=X_train, y=y_train, batch_size=NUM_BATCH, epochs=NUM_EPOCH, 
    verbose=2, validation_data=(X_test, y_test))

In [ ]:
from keras.applications import MobileNet
from keras.layers import Conv2D, Activation, BatchNormalization, Concatenate, AveragePooling2D, Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import pickle
import numpy as np

if __name__ == "__main__":
    # k=16の場合
    densenet = MobileNet
    # densenet.model.summary()

    # CIFAR-10の読み込み
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    # X_train = (X_train / 255.0).astype("float32")
    X_test = (X_test / 255.0).astype("float32")
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    densenet.train(X_train, y_train, X_test, y_test)